<a href="https://colab.research.google.com/github/Teitei011/Artificial-Intelligence/blob/master/CoronaBrasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import pandas as pd
import os

In [ ]:
os.mkdir("brazil")

In [ ]:
def determine_state_city_or_country(name):
    '''
        This is a hack that only work on this specific scenario
    '''
    if (len(name) == 2):
        return "state"
    elif(name == "Brazil"):
        return "Brazil"
    else:
        return "city"

Get data from a individual city/state

In [ ]:
def get_data(dataframe, name):
    index_ = determine_state_city_or_country(name)
    result_dataframe =  dataframe[dataframe[index_].isin([name])]
    
    result_dataframe.to_csv(f"{name}.csv", index=False)

 

In [ ]:
def separate_each_city_on_dataframe(dataframe):
    # set the index to be this and don't drop
    dataframe.set_index(keys=['city'], drop=False,inplace=True)
    
    # get a list of names
    city_names = dataframe['city'].unique().tolist()

    
    # now we can perform a lookup on a 'view' of the dataframe
    counter = 0
    amount_of_cities = len(city_names)
    for city in city_names:
        counter += 1
        print("Processing: {}  - {:.2f}% ...".format(city, counter*100/amount_of_cities))
        buffer_dataframe = dataframe.loc[dataframe.city==city]
        buffer_dataframe.drop(buffer_dataframe.index[0])
        buffer_dataframe.to_csv(f"brazil/{city}.csv", index=False)


def createADataframeToEachState(dataframe):
    all_states_name = dataframe["state"].unique()

    for state in all_states_name:
        print(f"Processing {state}...")
        buffer_dataframe = dataframe.loc[dataframe.state==state]
        buffer_dataframe.drop(buffer_dataframe.index[0])
        buffer_dataframe.to_csv(f"brazil/{state}.csv", index=False)

In [ ]:
def divideByTwoWhenPossible(array):
    for i in range(len(array)):
        try:
            array[i] = array[i] / 2
        except:
            pass
    return array


In [ ]:
def analyseCities(unprocessedDataset):
  city = unprocessedDataset.groupby(["municipio", "data"])[['casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos']].sum()
  city = city.rename(columns={"municipio": "city", "data": "date", 'casosAcumulado': 'cases', 'obitosAcumulado': 'deaths', 'casosNovos' : "daily cases", "obitosNovos": "daily deaths"})
  city.to_csv("brazil_cities.csv")


  cities = pd.read_csv("brazil_cities.csv")
  cities = cities.rename(columns={"municipio": "city", "data": "date"})


  cities.loc[:, "daily cases moving average"] = cities['daily cases'].rolling(window=7).mean() 
  cities.loc[:, "daily deaths moving average"] =   cities['daily deaths'].rolling(window=7).mean() 

  cities.loc[:,"sum_of_daily_cases_week"] = cities['daily cases'].rolling(window=7).sum() 
  cities.loc[:,"sum_of_daily_deaths_week"] = cities['daily deaths'].rolling(window=7).sum() 

  return cities

def analyseStates(unprocessedDataset):

  state = unprocessedDataset.groupby(["estado", "data"])[['casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos']].sum().apply(lambda x: divideByTwoWhenPossible(x))
  state = state.rename(columns={"estado": "state", "data": "date", 'casosAcumulado': 'cases', 'obitosAcumulado': 'deaths', 'casosNovos' : "daily cases", "obitosNovos": "daily deaths"})

  state.to_csv("brazil_states.csv")

  states = pd.read_csv("brazil_states.csv")

  states = states.rename(columns={"estado": "state", "data": "date"})


  states.loc[:, "daily cases moving average"] = states['daily cases'].rolling(window=7).mean() 
  states.loc[:, "daily deaths moving average"] =   states['daily deaths'].rolling(window=7).mean() 

  states.loc[:,"sum_of_daily_cases_week"] = states['daily cases'].rolling(window=7).sum() 
  states.loc[:,"sum_of_daily_deaths_week"] = states['daily deaths'].rolling(window=7).sum() 

  return states


In [ ]:

def main(file_name):
  tempo_inicial = time.time()
  unprocessedDataset = pd.read_csv(str(file_name), delimiter=";", error_bad_lines=False)
  
  cities = analyseCities(unprocessedDataset)
  states = analyseStates(unprocessedDataset)
    
  separate_each_city_on_dataframe(cities)
  createADataframeToEachState(states)

  print("It took {:.2f} minutes".format((time.time() - tempo_inicial)) )



In [ ]:
main("brazil.csv")

Processing: Abaetetuba  - 0.26% ...
Processing: Abel Figueiredo  - 0.53% ...
Processing: Abreulândia  - 0.79% ...
Processing: Acará  - 1.06% ...
Processing: Acrelândia  - 1.32% ...
Processing: Afuá  - 1.59% ...
Processing: Aguiarnópolis  - 1.85% ...
Processing: Alenquer  - 2.12% ...
Processing: Aliança do Tocantins  - 2.38% ...
Processing: Almas  - 2.65% ...
Processing: Almeirim  - 2.91% ...
Processing: Alta Floresta D'Oeste  - 3.17% ...
Processing: Altamira  - 3.44% ...
Processing: Alto Alegre  - 3.70% ...
Processing: Alto Alegre dos Parecis  - 3.97% ...
Processing: Alto Paraíso  - 4.23% ...
Processing: Alvarães  - 4.50% ...
Processing: Alvorada  - 4.76% ...
Processing: Alvorada D'Oeste  - 5.03% ...
Processing: Amajari  - 5.29% ...
Processing: Amapá  - 5.56% ...
Processing: Amaturá  - 5.82% ...
Processing: Anajás  - 6.08% ...
Processing: Anamã  - 6.35% ...
Processing: Ananindeua  - 6.61% ...
Processing: Ananás  - 6.88% ...
Processing: Anapu  - 7.14% ...
Processing: Angico  - 7.41% ...

fatal: remote origin already exists.
[master (root-commit) 90374a3] CSV added
 3459 files changed, 1135372 insertions(+)
 create mode 100644 AC.csv
 create mode 100644 AL.csv
 create mode 100644 AM.csv
 create mode 100644 AP.csv
 create mode 100644 Abadia dos Dourados.csv
 create mode 100644 Abaetetuba.csv
 create mode 100644 "Abaet\303\251.csv"
 create mode 100644 Abaiara.csv
 create mode 100644 "Abar\303\251.csv"
 create mode 100644 "Aba\303\255ra.csv"
 create mode 100644 Abel Figueiredo.csv
 create mode 100644 Abre Campo.csv
 create mode 100644 Abreu e Lima.csv
 create mode 100644 "Abreul\303\242ndia.csv"
 create mode 100644 Acaiaca.csv
 create mode 100644 Acajutiba.csv
 create mode 100644 Acarape.csv
 create mode 100644 "Acara\303\272.csv"
 create mode 100644 Acari.csv
 create mode 100644 "Acar\303\241.csv"
 create mode 100644 "Acau\303\243.csv"
 create mode 100644 Acopiara.csv
 create mode 100644 "Acrel\303\242ndia.csv"
 create mode 100644 Adamantina.csv
 create mode 100644 Adolfo